In [1]:
events = spark.read.json("/databricks-datasets/structured-streaming/events/")
events.write.format("delta").save("/mnt/delta/events")
spark.sql("CREATE TABLE events USING DELTA LOCATION '/mnt/delta/events/'")

Out[1]: DataFrame[]

In [2]:
%sql
SELECT * from events ORDER BY time DESC LIMIT 100;

action,time
Close,1469688499
Close,1469688448
Close,1469688335
Close,1469688216
Close,1469688129
Close,1469688021
Close,1469687980
Close,1469687968
Close,1469687965
Close,1469687960


In [3]:
from pyspark.sql.functions import *

parquet_path = "/tmp/sais_eu_19_demo/loans"

In [4]:
spark.read.format("parquet").load(parquet_path).createOrReplaceTempView("loans_parquet")

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-1591693898829615> in <module> 
 ----> 1 spark . read . format ( "parquet" ) . load ( parquet_path ) . createOrReplaceTempView ( "loans_parquet" ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in load (self, path, format, schema, **options) 
 176 self . options ( ** options ) 
 177 if isinstance ( path , basestring ) : 
 --> 178 return self . _df ( self . _jreader . load ( path ) ) 
 179 elif path is not None : 
 180 if type ( path ) != list : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 131 # Hide where the exception came from that shows a non-Pythonic 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 
 135 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 AnalysisException : Path does not exist: dbfs:/tmp/sais_eu_19_demo/loans;

In [5]:
import random
import os
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [6]:
def random_checkpoint_dir():
  return "/tmp/sais_eu_19_demo/chkpt/%s" % str(random.randint(0,1000))

states = ['CA', 'TX', 'NY', 'WA']

@udf(returnType=StringType())
def random_state():
  return str(random.choice(states))

#function to start a streaming query with a stream of randomly generated data and append to the parquet table

def generate_and_append_data_stream(table_format, table_path):
  
    stream_data = (spark.readStream.format("rate").option("rowsPerSecond", 5).load().withColumn("loan_id", 10000+col("value")) .withColumn("funded_amnt", col("funded_amnt")-(rand()*2000)).withColumn("addr_state", random-state()))
    
    query = (stream_data.writeStream.format(table_format).option("checkpointLocation", random_checkpoint_dir()).trigger(processingTime="10 seconds").start(table_path))
    
    return query

In [7]:
#added a comment
#added a comment2
#added a comment3